<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js112_SIRinfectionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [2]:
#@title js112_SIRinfectionModel / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023 Mitsuru Ikeuchi
#  home page: http://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.12.13 created,  last updated on 2024.01.07
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js112_SIRinfectionModel</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js112_SIRinfectionModel
//    Copyright(C) 2020-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2020.12.17 created, last updated on 2021.01.21
//    ver 0.0.1  2021.10.16 v1, last updated on 2021.10.16
//    ver 0.0.2  2021.11.08 v2, last updated on 2021.11.08
//    ver 0.0.3  2023.05.19 v3, last updated on 2023.09.12
//
// -------------------- SIR infection model - Finite Difference Method
//
//    S-I-R model
//
//      [S] --- beta*I ---> [I]--- gamma ---> [R]
//
//
//    S-I-R model -- differential equation
//      dS/dt = -beta*S*I
//      dI/dt = beta*S*I -gamma*I
//      dR/dt = gamma*I
//    where
//      S: susceptible
//      I: infectious
//      R: removed (recover + dead)
//      beta: infection rate (1/day)
//      gamma: recovery rate (1/day) ~ 1/(mean infected period)
//
//    basic reproduction number R0
//      R0 = beta*N/gamma, (N = S + I * R)
//     (R0 ~ 2~3 for influenza)  -- infection model
//
//
// --------------------
*/

const SIRinfectionModel = (function(){ // ====================  SIRinfectionModel Module  ====================

	const g_nMax = 360*3*20;			// array max
	const g_dt = 1.0/20.0;				// (day) time step g_dt

	let g_beta = 0.001;					// (1/day) infection rate
	let g_gamma = 0.1;					// (1/day) recovery rate
	let g_pN0 = 1000.0;					// (person) total = S+I+R at t=0
	let g_pI0 = 1.0;					// (person) infectius at t=0
	let g_pS0 = g_pN0 - g_pI0;			// (person) susceptible at t=0
	let g_pR0 = 0.0;					// (person) removed at t=0

	const g_pS = dim1( g_nMax );		// (person) susceptible
	const g_pI = dim1( g_nMax );		// (person) infectius
	const g_pR = dim1( g_nMax );		// (person) removed
	const g_pN = dim1( g_nMax );		// (person) total = S+E+I+R

	function dim1( n ) {
		return new Float64Array( n );
	}


	// --------------------  set SIR function  --------------------

	function setSIRfunc( v_pN0, v_pI0, v_beta, v_gamma ) {
		const nn=g_nMax, dt=g_dt;
		let sysTime = 0.0;

		g_beta = v_beta;
		g_gamma = v_gamma;
		g_pN0 = v_pN0;
		g_pI0 = v_pI0;
		g_pS0 = g_pN0 - g_pI0
		g_pR0 = 0.0;

		const beta = g_beta;
		const gamma = g_gamma;
		let pS = g_pS0;
		let pI = g_pI0;
		let pR = g_pR0;
		let pN = pS + pI + pR;

		for (let i=0; i<nn; i++) {
			g_pS[i] = pS;
			g_pI[i] = pI;
			g_pR[i] = pR;
			g_pN[i] = pS + pI + pR;

			//dS/dt = -beta*S*I
			const dSvdt = -beta*pS*pI;
			//dI/dt = beta*S*I -gamma*I
			const dIvdt = beta*pS*pI - gamma*pI;
			//dR/dt = gamma*I
			const dRvdt = gamma*pI;

			sysTime += dt;
			pS = pS + dSvdt*dt;
			pI = pI + dIvdt*dt;
			pR = pR + dRvdt*dt;
		}
	}


	// --------------------  utility  --------------------

	function maxInfectiusIndex() {
		const nn=g_nMax;

		let im = 0, m = g_pI[0];
		for(let i=0; i<nn; i++) {
			if (g_pI[i]>m) {
				m = g_pI[i];
				im = i;
			}
		}

		return im;
	}

	function  reproductionNumber() {
		return g_beta*g_pS[0]/g_gamma;
	}


	// --------------------  public  --------------------

	return {
		setSIRfunc:			setSIRfunc,	// setSIRfunc( v_pN0, v_pI0, v_beta, v_gamma )

		getSysParam:	function() { return [ g_nMax, g_pN0, g_dt, reproductionNumber() ]; },
		getS:			function(i) { return g_pS[i]; },
		getI:			function(i) { return g_pI[i]; },
		getR:			function(i) { return g_pR[i]; },
		getN:			function(i) { return g_pN[i]; },
		getSIRN:		function(i) { return [ g_pS[i], g_pI[i], g_pR[i], g_pN[i] ]; },
		getImax:		function() {  const iMax = maxInfectiusIndex(); return [ iMax, g_pI[iMax] ]; },
	};

})(); // ====================  SIRinfectionModel end  ====================


const js112 = (function(){ // ====================  js Module  ====================

	const theModule = SIRinfectionModel;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;					// = canvas.getContext('2d');

	let v_pN0 = 1000.0;			// (person) nPop = susceptible + infectious + removed
	let v_pI0 = 1.0;			// (person) infectious = 1
	let v_beta = 0.00025;		// infection rate (1/day/infectious)
	let v_gamma = 0.1;			// recovery rate (1/day) ~ 1/(mean infected period)

	let p_nMax, p_pN0, p_dt, p_reproductionNumber; // <-- theModule.getSysParam()

  let ssList = [];
  let iiList = [];
  let rrList = [];
  let nnList = [];


	let dispMode = 0; 			// no use
	let timeRange = 360;		// (day)
	let resetFlag = true;
	let outputFlag = false;

  let breakFlag = false;
  let dataFlag = true;

	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d', { willReadFrequently:true } );
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			outputFlag = false;
			theModule.setSIRfunc( v_pN0, v_pI0, v_beta, v_gamma );
			[ p_nMax, p_pN0, p_dt, p_reproductionNumber ] = theModule.getSysParam();
      dataFlag = true;
		}

		draw( ctx, dispMode, timeRange, outputFlag );

    if ( dataFlag ) setPopData();

		requestAnimationFrame(animate);
	}

  function setPopData() {
    ssList = [];
    iiList = [];
    rrList = [];
    nnList = [];
    for (let i=0; i<timeRange; i++) {
      const j = i*20;
      let pS, pI, pR, pN;
      [ pS, pI, pR, pN ] = theModule.getSIRN(j);
      ssList[i] = pS;
      iiList[i] = pI;
      rrList[i] = pR;
      nnList[i] = pN;
    }
	}


	//--------------------  draw  --------------------

	const gColor = { S:"#0088ff", I:"#ff0000", R:"#00ff00", N:"#ff00ff", text:"#888888", grid:"#666666" };

	function draw( ctx, dispMode, timeRange, outputFlag ) {
		const xBoxSize = 400, yBoxSize = 400, xp=60, yp=60;

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		if (dispMode==0) {
			plotSIR(ctx, xp,yp,timeRange);
		} else if (dispMode==1) {
			logPlotSIR(ctx, xp,yp,timeRange);
		}

		// caption
		const iMax = theModule.getImax()[0];
		let pS, pI, pR, pN; [ pS, pI, pR, pN ] = theModule.getSIRN(iMax);
		ctx.fillStyle = "#888888";
		ctx.fillText(
			`infectius max = ${pI.toFixed(1)} at ${(iMax*p_dt).toFixed(2)} (day), S+I+R = ${(pS+pI+pR).toFixed(2)}`,
			10, yCanvasSize-25);
		ctx.fillText(
			`beta = ${v_beta.toFixed(5)},  gamma = ${v_gamma.toFixed(2)},  R0 = ${p_reproductionNumber.toFixed(2)}`,
			10, yCanvasSize-5);

		//if ( outputFlag ) outputTextArea(1,timeRange);
	}

	function plotSIR(ctx, xp,yp,timeRange) {
		const mag = 300.0/p_pN0, m=Math.floor(timeRange/360/p_dt);

		//drawGrid(xp,yp,xLen,yLen,xDiv,yDiv,x0Str,xMaxStr,y0Str,yMaxStr,color)
		drawGrid(ctx, xp,yp,360,300,30,30,"0",timeRange+"(day)","0","1000", gColor.grid );

		drawText(ctx, "Susceptible", xp+20, yp-5, gColor.S );
		drawText(ctx, "Infectious", xp+120, yp-5, gColor.I );
		drawText(ctx, "Removed", xp+220, yp-5, gColor.R );
		drawText(ctx, "Population", xp+160, yp-25, gColor.N );
		drawText(ctx, "plot SIR", xp+100, yp+360, gColor.text );

		plotFunc( ctx, xp, yp, m, theModule.getS, mag, gColor.S );
		plotFunc( ctx, xp, yp, m, theModule.getI, mag, gColor.I );
		plotFunc( ctx, xp, yp, m, theModule.getR, mag, gColor.R );
		plotFunc( ctx, xp, yp, m, theModule.getN, mag, gColor.N );
	}

	function plotFunc(ctx, xp,yp,m,pFunc,mag,colr) {
		ctx.strokeStyle = colr;
		ctx.beginPath();
		for (let i=0; i<360; i++) {
			ctx.lineTo(xp+i, yp+300-mag*pFunc(i*m));
		}
		ctx.stroke();
	}

	function logPlotSIR(ctx, xp,yp,timeRange) {
		const m=Math.floor(timeRange/360/p_dt);

		//drawGrid(xp,yp,xLen,yLen,xDiv,yDiv,x0Str,xMaxStr,y0Str,yMaxStr,color)
		drawGrid(ctx, xp,yp,360,300,30,50,"0",timeRange+"(day)","10^0","10^3", gColor.grid );

		drawText(ctx, "Susceptible", xp+20, yp-5, gColor.S );
		drawText(ctx, "Infectious", xp+120, yp-5, gColor.I );
		drawText(ctx, "Removed", xp+220, yp-5, gColor.R );
		drawText(ctx, "Population", xp+160, yp-25, gColor.N );
		drawText(ctx, "log plot SIR", xp+100, yp+360, gColor.text );

		plotLog10Func(ctx, xp,yp,m,theModule.getS, gColor.S );
		plotLog10Func(ctx, xp,yp,m,theModule.getI, gColor.I );
		plotLog10Func(ctx, xp,yp,m,theModule.getR, gColor.R );
		plotLog10Func(ctx, xp,yp,m,theModule.getN, gColor.N );
	}

	function plotLog10Func(ctx, xp,yp,m,pFunc,colr) {
		ctx.strokeStyle = colr;
		ctx.beginPath();
		for (let i=0; i<360; i++) {
			let p = pFunc(i*m);
			if (p<=0.0) p = 0.1;
			const logp = Math.log10(p);
			if (logp>-0.2) ctx.lineTo(xp+i, yp+300-100.0*logp);
		}
		ctx.stroke();
	}

	function drawGrid(ctx, xp,yp,xLen,yLen,xDiv,yDiv,x0Str,xMaxStr,y0Str,yMaxStr,color) {
		ctx.strokeStyle = color;
		ctx.fillStyle = color;
		ctx.strokeRect(xp,yp,xLen,yLen);

		for (let i=0; i<xLen; i+=xDiv) {
			drawLine(ctx, xp+i,yp, xp+i, yp+yLen,color);
		}
		for (let j=0; j<yLen; j+=yDiv) {
			drawLine(ctx, xp,yp+yLen-j, xp+xLen, yp+yLen-j,color);
		}
		ctx.fillText(x0Str, xp-5, yp+yLen+20);
		ctx.fillText(xMaxStr, xp+xLen-xMaxStr.length*4, yp+yLen+20);
		ctx.fillText(y0Str, xp-y0Str.length*8-12, yp+yLen+5);
		ctx.fillText(yMaxStr, xp-yMaxStr.length*8-12, yp+5);
	}

	 function drawLine( ctx, x1, y1, x2, y2, color ) {
	 	ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}

	function drawText( ctx, txt, x, y, color ) {
		ctx.fillStyle = color;
		ctx.fillText(txt, x, y);
	}


	// --------------------  data output  --------------------

	function outputTextArea(outputSW,timeRange) {
		// outputSW:  0:textarea clear,  1:output text data
		outputFlag = false;

		let str;

		if (outputSW==0) {
			str = "";

		} else if (outputSW==1) {
			str = `beta = ${v_beta.toFixed(5)}, gamma = ${v_gamma.toFixed(2)}, R0 = ${p_reproductionNumber.toFixed(2)}\n`;
			str += `S0 = ${theModule.getS(0).toFixed(1)}, I0 = ${theModule.getI(0).toFixed(1)}\n\n`;
			str += `day,  S(t),  I(t),  R(t) \n`;
			for (let i=0; i<timeRange; i++) {
				const j = i*20;
				let pS, pI, pR, pN; [ pS, pI, pR, pN ] = theModule.getSIRN(j);
				str += ` ${i},  ${pS.toFixed(1)},  ${pI.toFixed(1)},  ${pR.toFixed(1)}\n`;
			}
		}
		document.getElementById("outputText").innerHTML = str;
	}


	// --------------------  dom control  --------------------

	function setBeta() {
		const b = 0 + document.getElementById("range_beta").value;
		v_beta = 0.00001*b;
		document.getElementById("text_beta").innerHTML = " " + v_beta.toFixed(5);
		resetFlag = true;
	}

	function setGamma() {
		const g = 0 + document.getElementById("range_gamma").value;
		v_gamma = 0.005*g;
		document.getElementById("text_gamma").innerHTML =
			" " + v_gamma.toFixed(2) + " (1/ " + (1.0/v_gamma).toFixed(1) + " day)";
		resetFlag = true;
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setTimeRange() {
		timeRange = setTimeRange.range[ 0 + document.getElementById("slct_timeRange").selectedIndex ];
	}
	setTimeRange.range = [180,360,720,1080];

	function getData() {
		outputFlag = true;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetBeta( beta ) {
		v_beta = beta;
		document.getElementById("text_beta").innerHTML = " " + v_beta.toFixed(5);
		resetFlag = true;
  }

  function pysetGamma( gamma ) {
		v_gamma = gamma;
		document.getElementById("text_gamma").innerHTML =
			" " + v_gamma.toFixed(2) + " (1/ " + (1.0/v_gamma).toFixed(1) + " day)";
		resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ v_pN0, v_pI0, v_beta, v_gamma, p_reproductionNumber ];
  }

  function pygetPopData() {
    dataFlag = false;
    return [ ssList, iiList, rrList, nnList ];
  }


	// --------------------  public  --------------------

	return {
		main:				main,				// main()

		// dom control
		setBeta:			setBeta,			// setBeta()
		setGamma:			setGamma,			// setGamma()

		setDispMode:		setDispMode,		// setDispMode()
		setTimeRange:		setTimeRange,		// setTimeRange()
		getData:			getData,			// getData()

    breakLoop: breakLoop, // breakLoop();
    pysetBeta: pysetBeta, // pysetBeta( beta )
    pysetGamma: pysetGamma, // pysetGamma( gamma )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ pN0, pI0, beta, gamma, R0 ]
    pygetPopData, pygetPopData, //() :return [ ssList, iiList, rrList, nnList ]
	};

})(); // ====================  js112 module end  ====================


const js = js112;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js112] SIR infection model - solve finite difference equations</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas>
<br>

<span>theme: S-I-R infection model</span>
<br>

<label>beta (infection rate ) =</label>
<input type="range" id="range_beta" min="1" max="100" value="25" step="1"
style="width:200px" oninput="js.setBeta()">
<label id="text_beta"> 0.00025</label> (1/day/infectious)
<br>

<label>gamma (recovery rate)=</label>
<input type="range" id="range_gamma" min="1" max="100" value="20" step="1"
style="width:200px" oninput="js.setGamma()">
<label id="text_gamma"> 0.10 (1/ 10.0 day)</label>
<br>

<label>dispMode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option selected>plot SIR</option><option >log plot SIR</option></select>
    <span style="margin-right: 50px;"></span>
<label>time range:</label>
<select id="slct_timeRange" onChange="js.setTimeRange()">
<option >0 - 180</option><option selected>0 - 360</option><option >0 - 720</option>
<option >0 - 1080</option></select>
<br>

<!--
<button id="getData_button" onClick="js.getData()">output text data</button>
<br>

<textarea id="outputText" rows="6" cols="60" disabled></textarea>
<br>
-->

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get S-I-R population

import time

# exec html-js code
exec_html_js()
print("-- start --")

print("-- get system parameter --")
[ pN0, pI0, beta, gamma, R0 ] = eval_js( 'js.pygetData({})'.format(0) )
print(f' N = {pN0}, I0 = {pI0}, beta = {beta:>10.6f}, gamma = {gamma:>10.6f}, reproduction number R0 = {R0:6.2f} ')

# get SIR data
print("-- get SIR data --")
[ ssList, iiList, rrList, nnList ] = eval_js('js.pygetPopData()')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- simulator stop --")

print(f'len(ssList) = {len(ssList)}, len(iiList) = {len(iiList)}, len(rrList) = {len(rrList)}, len(nnList) = {len(nnList)}, ')
print("-- end --")

In [ ]:
# plot S-I-R

import matplotlib.pyplot as plt

plt.plot(ssList, 'b-')
plt.plot(iiList, 'r-')
plt.plot(rrList, 'g-')
plt.title('S-I-R Model')
plt.xlabel('time( day )')
plt.ylabel('population')
plt.show()

In [ ]:
# def beta_to_SIR(beta, gamma), plot_SIR(S, I, R)

import numpy as np
import matplotlib.pyplot as plt

def beta_to_SIR(beta=0.00025, gamma=0.10):
  # exec html-js code
  exec_html_js()

  # set beta,gamma
  eval_js(f'js.pysetBeta( {beta} )')
  eval_js(f'js.pysetGamma( {gamma} )')
  [ pN0, pI0, beta, gamma, R0 ] = eval_js( 'js.pygetData({})'.format(0) )
  [ ssList, iiList, rrList, nnList ] = eval_js('js.pygetPopData()')
  S = np.array(ssList)
  I = np.array(iiList)
  R = np.array(rrList)
  N = np.array(nnList)

  # end
  eval_js( 'js.breakLoop()' )

  return [ pN0, pI0, beta, gamma, R0, S, I, R, N ]

def plot_SIR(S,I,R):
  plt.plot(S, 'b-')
  plt.plot(I, 'r-')
  plt.plot(R, 'g-')
  plt.title('S-I-R Model')
  plt.xlabel('time( day )')
  plt.ylabel('population')
  plt.show()

# test
[ pN0, pI0, beta, gamma, R0, S, I, R, N ] = beta_to_SIR(beta=0.0003)
print(f' N = {pN0}, I0 = {pI0}, beta = {beta:>10.6f}, gamma = {gamma:>10.6f}, reproduction number R0 = {R0:6.2f} ')

plot_SIR(S, I, R)

In [ ]:
# beta = 0.00025, gamma = 0.1 case

[ pN0, pI0, beta, gamma, R0, S, I, R, N ] = beta_to_SIR(beta=0.00025,gamma=0.1)
print(f' N = {pN0}, I0 = {pI0}, beta = {beta:>10.6f}, gamma = {gamma:>10.6f}, reproduction number R0 = {R0:6.2f} ')
plot_SIR(S, I, R)

In [ ]:
# save data

import numpy as np

print("-- save data : np.array([ S, I, R, N ]) --")
np_data = np.array([ S, I, R, N ])

np.save('js112_data.npy', np_data)

print("-- saved data shape :", np_data.shape, " --")

In [ ]:
# load SIR data

import numpy as np

np_data = np.load('js112_data.npy')
print("-- loaded data shape :", np_data.shape, " --")
S=np_data[0]
I=np_data[1]
R=np_data[2]
N=np_data[3]

plot_SIR(S, I, R)

In [ ]:
# prompt: plot S,I,R with plotly

import plotly.graph_objects as go

def plot_SIR_plotly(S, I, R):
  fig = go.Figure()
  fig.add_trace(go.Scatter(y=S, mode='lines', name='Susceptible'))
  fig.add_trace(go.Scatter(y=I, mode='lines', name='Infected'))
  fig.add_trace(go.Scatter(y=R, mode='lines', name='Recovered'))
  fig.update_layout(title='S-I-R Model',
                    xaxis_title='Time (days)',
                    yaxis_title='Population')
  fig.show()

# Assuming S, I, R are already defined from the previous code
plot_SIR_plotly(S, I, R)